In [1]:
from liblibra_core import *
from libra_py import *
import math
import py3Dmol

/home/alexey/miniconda2/envs/libra/lib/python2.7/site-packages/ipykernel_launcher.py:1: RuntimeWarning: to-Python converter for std::vector<std::complex<double>, std::allocator<std::complex<double> > > already registered; second conversion method ignored.
  """Entry point for launching an IPython kernel.
/home/alexey/miniconda2/envs/libra/lib/python2.7/site-packages/ipykernel_launcher.py:1: RuntimeWarning: to-Python converter for std::vector<std::vector<int, std::allocator<int> >, std::allocator<std::vector<int, std::allocator<int> > > > already registered; second conversion method ignored.
  """Entry point for launching an IPython kernel.
/home/alexey/miniconda2/envs/libra/lib/python2.7/site-packages/ipykernel_launcher.py:1: RuntimeWarning: to-Python converter for boost::python::detail::container_element<std::vector<std::vector<int, std::allocator<int> >, std::allocator<std::vector<int, std::allocator<int> > > >, unsigned long, boost::python::detail::final_vector_derived_policies<std:

In [2]:
def fold_coords1(R,H):
    ndof = R.num_of_rows
    nsteps = R.num_of_cols
    nat = ndof/3
    
    res = MATRIX(ndof, nsteps)
    
    for step in xrange(nsteps):
        r_in = VECTORList()
        for at in xrange(nat):
            r_in.append(VECTOR(R.get(3*at, step), R.get(3*at+1, step), R.get(3*at+2, step)))
        
        #r_out = apply_pbc(H, r_in, boost::python::list t){


            
params = {"nsteps":100, "nperiods":1, "scale":250.0, "visualize":0, "verbosity":0, "prefix":"_Si8_cov_flag0"}

R, V, A, M, E = QE_methods.read_md_data("x0.xml")
#R.scale(-1,-1, 1.0/units.Angst)
C = QE_methods.read_md_data_cell("x0.xml")

print C.get(0,0), C.get(1,0), C.get(2,0)
print C.get(3,0), C.get(4,0), C.get(5,0)
print C.get(6,0), C.get(7,0), C.get(8,0)

H = MATRIX3x3()
H.xx = C.get(0,0);  H.xy = C.get(0,1);  H.xz = C.get(0,2);
H.yx = C.get(1,0);  H.yy = C.get(1,1);  H.yz = C.get(1,2);
H.zx = C.get(2,0);  H.zy = C.get(2,1);  H.zz = C.get(2,2);

#alat = 10.26253537973
tH = H.T()  #* alat # * units.Angst

fR = fold_coords(R, tH, "abc")

10.2625353797 0.0 0.0
0.0 10.2625353797 0.0
0.0 0.0 10.2625353797


### Method 1:

Strachan, A. Normal Modes and Frequencies from Covariances in Molecular Dynamics 
    or Monte Carlo Simulation. J. Chem. Phys. 2003, 120, 1-4.
    
In the following 2 calls *res* is a tuple (w, w_inv_cm, U_v,  w2, w2_inv_cm, U_a)

In [3]:
params.update({"cov_flag":0})
res00 = normal_modes.compute_cov( R, V, A, M, E, params)

Another option is to use data centered around the averages (fluctuations)

In [4]:
params.update({"cov_flag":1})
res01 = normal_modes.compute_cov( R, V, A, M, E, params)


### Method 2:
Pereverzev, A.; Sewell, T. D. Obtaining the Hessian from the Force Covariance Matrix:
    Application to Crystalline Explosives PETN and RDX. J. Chem. Phys. 2015, 142, 134110.
    
In the following 2 calls *res* is a tuple (w_a, w_inv_cm, U_a)
Again, without centering:

In [5]:
params.update({"cov_flag":0})
res20 = normal_modes.compute_cov2( R, A, M, E, 300.0, params)

With the centering:

In [6]:
params.update({"cov_flag":1})
res21 = normal_modes.compute_cov2( R, A, M, E, 300.0, params)

Now, we can select which normal modes to visualize:

In [7]:
mode = 10

# Method 1
xyz00v = normal_modes.get_xyz(E,R, M, res00[2], mode)  # velocity covariance
xyz00a = normal_modes.get_xyz(E,R, M, res00[5], mode)  # acceleration covariance
xyz01v = normal_modes.get_xyz(E,R, M, res01[2], mode)  # velocity fluctuation covariance
xyz01a = normal_modes.get_xyz(E,R, M, res01[5], mode)  # acceleration fluctuation covariance

# Method 2
xyz20f = normal_modes.get_xyz(E,R, M, res20[2], mode)  # force covariance
xyz21f = normal_modes.get_xyz(E,R, M, res21[2], mode)  # force fluctuation covariance


view = py3Dmol.view(width=800,height=400, linked=False,viewergrid=(3,2))
view.addModel(xyz00v,'xyz',{'vibrate': {'frames':10,'amplitude':500}}, viewer=(0,0))
view.addModel(xyz01v,'xyz',{'vibrate': {'frames':10,'amplitude':500}}, viewer=(0,1))

view.addModel(xyz00a,'xyz',{'vibrate': {'frames':10,'amplitude':500}}, viewer=(1,0))
view.addModel(xyz01a,'xyz',{'vibrate': {'frames':10,'amplitude':500}}, viewer=(1,1))

view.addModel(xyz20f,'xyz',{'vibrate': {'frames':10,'amplitude':500}}, viewer=(2,0))
view.addModel(xyz21f,'xyz',{'vibrate': {'frames':10,'amplitude':500}}, viewer=(2,1))

view.setBackgroundColor('0xeeeeee')
view.setStyle({'sphere':{}})
view.animate({'loop': 'backAndForth'})
view.zoomTo()
view.show()

print xyz00v

You appear to be running in JupyterLab. You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

  8
Comment
Si  0.004  0.003  0.001  0.001  -0.000  0.001
Si  -0.004  5.138  5.144  0.001  0.001  0.001
Si  5.130  5.134  0.002  0.002  -0.000  -0.001
Si  5.137  -0.003  5.128  -0.000  0.000  -0.001
Si  7.699  2.563  7.678  -0.001  -0.001  0.001
Si  2.557  2.556  2.576  -0.001  0.001  0.000
Si  2.565  7.704  7.695  0.000  0.001  0.001
Si  7.699  7.693  2.564  -0.001  -0.001  -0.002



In [8]:
si8="""8

Si 0.000000000000000 0.000000000000000 0.000000000000000
Si 0.000000000000000 5.131267690524859 5.131267690524859
Si 5.131267690524859 5.131267690524859 0.000000000000000
Si 5.131267690524859 0.000000000000000 5.131267690524859
Si 7.696901534842424 2.565633844317566 7.696901534842424
Si 2.565633844317566 2.565633844317566 2.565633844317566
Si 2.565633844317566 7.696901534842424 7.696901534842424
Si 7.696901534842424 7.696901534842424 2.565633844317566"""

view1 = py3Dmol.view(width=800,height=400)
view1.addModel(si8,'xyz',{'vibrate': {'frames':10,'amplitude':500}})
view1.setBackgroundColor('0xeeeeee')
view1.setStyle({'sphere':{}})
view1.animate({'loop': 'backAndForth'})
view1.zoomTo()
view1.show()

print si8
print xyz00v

You appear to be running in JupyterLab. You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

8

Si 0.000000000000000 0.000000000000000 0.000000000000000
Si 0.000000000000000 5.131267690524859 5.131267690524859
Si 5.131267690524859 5.131267690524859 0.000000000000000
Si 5.131267690524859 0.000000000000000 5.131267690524859
Si 7.696901534842424 2.565633844317566 7.696901534842424
Si 2.565633844317566 2.565633844317566 2.565633844317566
Si 2.565633844317566 7.696901534842424 7.696901534842424
Si 7.696901534842424 7.696901534842424 2.565633844317566
  8
Comment
Si  0.004  0.003  0.001  0.001  -0.000  0.001
Si  -0.004  5.138  5.144  0.001  0.001  0.001
Si  5.130  5.134  0.002  0.002  -0.000  -0.001
Si  5.137  -0.003  5.128  -0.000  0.000  -0.001
Si  7.699  2.563  7.678  -0.001  -0.001  0.001
Si  2.557  2.556  2.576  -0.001  0.001  0.000
Si  2.565  7.704  7.695  0.000  0.001  0.001
Si  7.699  7.693  2.564  -0.001  -0.001  -0.002

